# Level 1: Foundational RAG

This tutorial presents an example of executing queries with foundational (i.e., non-agentic) RAG in Llama Stack. It shows how the APIs provided by Llama Stack can be used to directly control and invoke all RAG stages, including indexing, retrieval and inference. 
For an agentic RAG tutorial, please refer to [Level3_agentic_RAG.ipynb](demos/rag_agentic/notebooks/Level3_agentic_RAG.ipynb).

## Overview

This tutorial covers the following steps:
1. Connecting to a llama-stack server.
2. Indexing a collection of documents in a vector DB for later retrieval.
3. Executing the built-in RAG tool to retrieve the document chunks relevant to a given query.
4. Using the retrieved context to answer user queries during the inference step.


## Prerequisites

Before starting, ensure you have a running instance of the Llama Stack server (local or remote) with at least one preconfigured vector DB. For more information, please refer to the corresponding [Llama Stack tutorials](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html).

## 1. Setting Up the Environment
- Import the necessary libraries.
- Define the settings for the RAG pipeline, including the Llama Stack server URL, inference and document ingestion parameters.
- Initialize the connection to the server.

In [3]:
import os
import uuid

from llama_stack_client import Agent, AgentEventLogger, RAGDocument, LlamaStackClient
from llama_stack_client.types.shared.content_delta import TextDelta, ToolCallDelta

# the server endpoint
LLAMA_STACK_SERVER_URL = "http://localhost:8321"

# inference settings
MODEL_ID = "ibm-granite/granite-3.2-8b-instruct"
SYSTEM_PROMPT = "You are a helpful assistant. "
TEMPERATURE = 0.0
TOP_P = 0.95

# RAG settings
VECTOR_DB_EMBEDDING_MODEL = "all-MiniLM-L6-v2"
VECTOR_DB_EMBEDDING_DIMENSION = 384
VECTOR_DB_CHUNK_SIZE = 512

# For this demo, we are using Milvus Lite, which is our preferred solution. Any other Vector DB supported by Llama Stack can be used.
VECTOR_DB_PROVIDER_ID = 'milvus'

# initialize the inference strategy
if TEMPERATURE > 0.0:
    strategy = {"type": "top_p", "temperature": TEMPERATURE, "top_p": TOP_P}
else:
    strategy = {"type": "greedy"}
    
# initialize the document collection to be used for RAG
vector_db_id = f"test_vector_db_{uuid.uuid4()}"
    
# initialize the server connection
client = LlamaStackClient(base_url=os.environ.get("LLAMA_STACK_ENDPOINT", LLAMA_STACK_SERVER_URL))

## 2. Indexing the Documents
- Initialize a new document collection in the target vector DB. All parameters related to the vector DB, such as the embedding model and dimension, must be specified here.
- Provide a list of document URLs to the RAG tool. Llama Stack will handle fetching, conversion and chunking of the documents' content.

In [6]:
# define and register the document collection to be used
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=VECTOR_DB_EMBEDDING_MODEL,
    embedding_dimension=VECTOR_DB_EMBEDDING_DIMENSION,
    provider_id=VECTOR_DB_PROVIDER_ID,
)

# ingest the documents into the newly created document collection
urls = [
    ("https://www.openshift.guide/openshift-guide-screen.pdf", "application/pdf"),
    ("https://www.cdflaborlaw.com/_images/content/2023_OCBJ_GC_Awards_Article.pdf", "application/pdf"),
]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=url,
        mime_type=url_type,
        metadata={},
    )
    for i, (url, url_type) in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=VECTOR_DB_CHUNK_SIZE,
)

## 3. Executing Queries via the Built-in RAG Tool
- Directly invoke the RAG tool to query the vector DB we ingested into at the previous stage.
- Construct an extended prompt using the retrieved chunks.
- Query the model with the extended prompt.
- Output the reply received from the model.

In [24]:
queries = [
    "How to install OpenShift?",
    "Are employees based in California eligible for remote work?",
]

for prompt in queries:
    print(f"\nUser> {prompt}")
    
    # RAG retrieval call
    rag_response = client.tool_runtime.rag_tool.query(content=prompt, vector_db_ids=[vector_db_id])

    # the list of messages to be sent to the model must start with the system prompt
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]

    # construct the actual prompt to be executed, incorporating the original query and the retrieved content
    prompt_context = rag_response.content
    extended_prompt = f"Please answer the given query using the context below.\n\nCONTEXT:\n{prompt_context}\n\nQUERY:\n{prompt}"
    messages.append({"role": "user", "content": extended_prompt})

    # use Llama Stack inference API to directly communicate with the desired model
    response = client.inference.chat_completion(
        messages=messages,
        model_id=MODEL_ID,
        sampling_params={
            "strategy": strategy,
        },
        stream=True,
    )
    
    # print the response
    print("inference> ", end='')
    for chunk in response:
        response_delta = chunk.event.delta
        if isinstance(response_delta, TextDelta):
            print(response_delta.text, end='')
        elif isinstance(response_delta, ToolCallDelta):
            print(response_delta.tool_call, end='')


User> How to install OpenShift?
inference> To install OpenShift, you can follow these steps:

1. Download and install the OpenShift CLI (Command Line Interface) tool from the official Red Hat website.
2. Create a new project in your OpenShift cluster using the `oc new-project` command.
3. Install the OpenShift client tools on your system by running the `oc setup` command.
4. Log in to your OpenShift cluster using the `oc login` command with your credentials.
5. Create a new application in your project using the `oc new-app` command, specifying the image URL of the container you want to deploy.
6. Once the application is created, you can access it through the OpenShift web console or by running the `oc expose` command.

Alternatively, you can also use the odo tool provided by Red Hat to create and deploy applications on OpenShift. The odo tool allows you to create applications using "Devfiles," which are YAML files that contain information about your application's programming language,

## Key Takeaways
This tutorial demonstrates how to set up and use the built-in RAG tool for ingesting user-provided documents in a vector DB and later utilizing them during inference via direct retrieval. Please check out our [complementary tutorial]([Level3_agentic_RAG.ipynb](demos/rag_agentic/notebooks/Level3_agentic_RAG.ipynb) for an agentic RAG example.